## Specific conductance, dissolved load, and calcite saturation state

Specific conductance (SpC) is an indicator of the dissolved load in the water. At our site, we expect most of that dissolved load to be from carbonate minerals. However, it could also indicate pollution. In that case, other ions, such as Chloride (which we measured) could also be high.

#### Questions

- How does dissolved load change from one part of the system to another? What processes might add or remove it?
- Is there any evidence that these differences among sites are a function of polution, or could it just be differences in dissolved carbonate?
- Is the water undersaturated, saturated, or supersaturated with respect to calcite? How does this vary throughout the system?
- What are the long-term patterns in SpC observed at the site, and how do your observations fit into them?
- Does evolution of SpC along the sinking stream show any evidence of precipitation or dissolution, or can the changes in SpC be explained purely by mixing of waters from the springs and stream?

#### Read in water quality data

In [ ]:
# Read in the water quality data from the spreadsheet
from pandas import read_csv
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# Make a boxplot of SpC measurements categorized by type of site
# (spring, epikarst, stream, well)



### SpC and pollution

In [ ]:
# To examine whether there is any relationship between SpC and
# pollution, make a scatter plot of SpC vs Chloride.



### Calcite Saturation

To examine calcite saturation, we will use functions from a Python package called *Olm*, which was written to conduct basic water chemistry analysis in karst settings. First we need to install this package (if you haven't already done so). We can install using the following command.

In [ ]:
!pip install olm-karst

To see more about what *Olm* can do, check out the Github repository here:

https://github.com/CovingtonResearchGroup/olm/tree/master

Or the example tutorials here:

https://github.com/CovingtonResearchGroup/olm-examples

We will use *Olm* here to estimate the saturation state of the water with respect to calcite (the main component of limestone). These calcualtions assume that the chemistry is dominated by ions in the $H_2O - CO_2 - CaCO_3$ system. If we have a substantial component from other ions (for example from pollution), then these calculations will be inaccurate.

The first step is to estimate the dissolved Calcium concentration from our SpC values. This could be done using a global relationship which is provided in olm.general.CaFromCond(). However, we can do better, since we have a set of Ca concentrations and SpC measurements from Savoy.

In [ ]:
# This csv contains SpC (uS/cm) and Ca (mg/L) values from water samples at Savoy.
savoy_chem = read_csv('data/savoy_chem.csv')
savoy_chem

# Note, the pH and temp data here are fabricated (all the same value).
# We will just use them to illustrate the method of saturation
# ratio calculation below.

In [ ]:
# Let's plot the relationship
plt.plot(savoy_chem['SpC'], savoy_chem['Ca'], 'o')
plt.xlabel('Specific conductance $\mu$S/cm')
plt.ylabel('Ca concentration (mg/L)')

You can see that there is a relatively good linear relationship between Ca and SpC. Therefore, we can use this relationship to estimate what Ca would be, just from our SpC measurements in the field. To do this, we need to fit a line.

In [ ]:
from scipy.stats import linregress

slope, intercept, r, p, se = linregress(savoy_chem['SpC'], savoy_chem['Ca'])


In [ ]:
# We can plot this best fit on top of the data like this
plt.plot(savoy_chem['SpC'], savoy_chem['Ca'], 'o')
plt.xlabel('Specific conductance $\mu$S/cm')
plt.ylabel('Ca concentration (mg/L)')

# Create an array of SpC values to make our line, picking an appropriate range
SpC_model = np.linspace(100,340, 5) 
Ca_model = slope*SpC_model + intercept
plt.plot(SpC_model, Ca_model, '-')


In [ ]:
# Now, we can use values of SpC, along with the slope and
# intercept derived above, to estimate Ca. Here we will 
# illustrate this with the values above. However, you should
# do this below with the SpC values you collected in the field.

Ca_mg_L = slope*savoy_chem.SpC + intercept

Once we have Ca, we can use *Olm* to do the hard work of estimating calcite saturation state. It can approximate the chemistry of a solution using Ca and pH using the process below.

In [ ]:
# First, we need to convert Ca into mol/L.
from olm.general import mgL_to_molL

# We just have to provide the concentration and tell it which ion we have
Ca_mol_L = mgL_to_molL(Ca_mg_L, 'Ca')

In [ ]:
# This is the function we use to calculation the properties
# of the solution.
from olm.calcite import solutionFrompHCaRelaxed

pH_obs = savoy_chem.pH # use our fabricated pH values as example
T_C_obs = savoy_chem.T_C # use our fabricated temp values as example

solutions = solutionFrompHCaRelaxed(Ca_mol_L, pH_obs, T_C=T_C_obs)




With *Olm*, many types of calculations can be done with `solution` objects. Here we will calculate the equilibrium concentration of Ca with respect to calcite.

In [ ]:
from olm.calcite import concCaEqFromSolution

#This estimates the equilibrium Ca concentration in mol/L
Ca_eq = concCaEqFromSolution(solutions)


One way to express saturation state is using the 'saturation ratio,' ${\rm R=Ca/Ca_{eq}}$. The solution is saturated if $R=1$, undersaturated if $R<1$, and supersaturated if $R>1$.

In [ ]:
R = Ca_mol_L/Ca_eq
R

In [ ]:
# One way to visualize this would be with a histogram, since we
# have lots of values.

plt.hist(R,15) # Note: this is a slightly different way to make histograms, using matplotlib rather than pandas
plt.xlabel('Saturation ratio for Calcite')
plt.ylabel('Number per bin')

Now make similar calculations using your field data and the method demonstrated above. What is the saturation state at your field sites? In addition to a histogram, you might consider making a boxplot of saturation ratio at different types of sites. You can assume that (Ca) is well-approximated from SpC using the fit parameters from the historical chemical dataset above.

## Long-term SpC records

In [ ]:
# There are long-term SpC records from Langle
# and Copperhead Springs in 'data/Langle.csv' and
# 'data/Copperhead.csv' 

# Read in these data and plot up the time series of
# SpC at Langle and Copperhead. You can use
# these longer records to get a bigger context for the
# SpC data you collected yesterday.

# The Cond values corrected using spot measurements and adjusted to 25 C are
# in the column named Cond_corr.



## SpC as a tracker of mixing and reactions

One of the questions above is whether changes in SpC within the sinking stream are only indicative of conservative mixing (no reactions) or whether there are some reactions (most likely calcite precipitation or dissolution) that cause changes in SpC along the stream. To really answer this question, you will also need discharge data. However, you can get a start by making the *assumption* of conservative mixing and using it to calculate ratios of discharge in the main sinking stream and specific inputs. Note this will only work if some infeeding streams from the epikarst springs have SpC that is different from the main stream.

Under conservative mixing, from conservation of mass, we get:

$$C_{\rm tot} Q_{\rm tot} = C_1 Q_1 + C_2 Q_2 $$

where $C_1$ and $C_2$ are the concentrations of some ion (or all ions) in two streams that flow together, and $Q_1$ and $Q_2$ are the discharges in those streams. $C_{\rm tot}$ and $Q_{\rm tot}$ are the concentration and discharge of the combined flow.

Since SpC is proportional to the concentration of all dissolved ions, we can use it in place of the $C$'s in this equation. 

We also know that $Q_{\rm tot}=Q_1 + Q_2$, since these two streams join to make the total flow. We can use this equation to either $Q_2$ from the equation, and with some algebra can derive the following relationship:

$$ \frac{Q_1}{Q_{\rm tot}} = \frac{(C_{\rm tot} - C_2)}{(C_1 - C_2)} $$

If we let $Q_1$ be the flow coming in from one of the epikarst springs, then under an assumption of conservative mixing, we could use this equation to calculation the percentage of the flow that was being contributed from the spring. If we have discharge measurements from the spring and main stream, then we could compare.

If these the discharge ratio calculated from SpC doesn't match the discharge measurements, then either there is some sort of reaction going on that changes dissolved load, or our discharge measurements are in error.

In [ ]:
# Examine whether any of the spring junctions have sufficiently different SpC
# to try this mixing model. If so, calculate the percentage of flow contributed
# from that spring assuming conservative mixing. How does this compare to measured
# discharges?